<a href="https://colab.research.google.com/github/movie112/INU-DILAB/blob/main/dacon/titanic/titanic_deep_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [재난] 타이타닉: 누가 살아남았을까?   
- 참고: [blog_Titanic 생존자 예측 심화편#2](https://blog.naver.com/fkdldjs60/221925696786)
- ML model의 parameter 값 설정
  - 파라미터 값에 따라 ML 성능 변화
---
최적의 parameter 찾는 방법
- Grid Search 방법
  - 파라미터 값을 임의의 값들로 묶어 모든 경우의 수로 조합
  - 빠름
- Random Search 방법
  - 파라미터 값을 일정 범위로 설정해 무작위로 조합
  - 더 좋은 파라미터 도출하지만 시간 소요

In [3]:
import pandas as pd
import seaborn as sns

In [4]:
cd drive/MyDrive/data/titanic

/content/drive/MyDrive/data/titanic


In [5]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [6]:
train['Sex_encode'] = train['Sex'].replace('male', 0).replace('female', 1)
test['Sex_encode'] = train['Sex'].replace('male', 0).replace('female', 1)

In [7]:
train['Embarked_Q'] = train['Embarked'] == 'Q'
train['Embarked_S'] = train['Embarked'] == 'S'
train['Embarked_C'] = train['Embarked'] == 'C'

In [8]:
test['Embarked_Q'] = test['Embarked'] == 'Q'
test['Embarked_S'] = test['Embarked'] == 'S'
test['Embarked_C'] = test['Embarked'] == 'C'

In [9]:
test['Family'] = test['SibSp'] + test['Parch'] +1
train['Family'] = train['SibSp'] + train['Parch'] +1

In [10]:
def Family_type(Family):
  if Family == 1:
    return 'Single'
  elif Family > 1 and Family < 5:
    return 'Nuclear'
  else:
    return 'Big'

In [11]:
train['Family_type'] = train['Family'].apply(Family_type)
test['Family_type'] = test['Family'].apply(Family_type)

In [12]:
train['Single'] = train['Family_type'] == 'Single'
train['Nuclear'] = train['Family_type'] == 'Nuclear'
train['Big'] = train['Family_type'] == 'Big'
test['Single'] = test['Family_type'] == 'Single'
test['Nuclear'] = test['Family_type'] == 'Nuclear'
test['Big'] = test['Family_type'] == 'Big'

In [13]:
test['Fare'] = test['Fare'].fillna(0)

In [14]:
features = ['Pclass', 'Sex_encode', 'Embarked_Q', 'Embarked_S', 'Embarked_C', 'Single', 'Nuclear', 'Big']

In [15]:
x_train=train[features]
x_test=test[features]
y_train=train['Survived']

#### Grid Search
- 조합할 파라미터 값의 후보군 설정
  - max_depth[후보 값들]

In [20]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
parameters = {
    'max_depth':[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'max_features':[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
}
GS = GridSearchCV(DecisionTreeClassifier(), param_grid = parameters, cv=10, scoring = 'accuracy')
GS.fit(x_train, y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
      

#### 후보 중 best 파라미터 값 출력: .best_params_

In [21]:
GS.best_params_

{'max_depth': 4, 'max_features': 0.6}

In [22]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(max_depth=3, max_features=0.8)
model

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=3, max_features=0.8, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [24]:
from sklearn.model_selection import cross_val_score
score = cross_val_score(model, x_train, y_train, scoring = 'accuracy', cv=10).mean()
score

0.8103245942571785

---
## Ramdom Search
- float 실수형에는 range 이용할 수 없어 uniform 이용


In [26]:
from scipy import stats
from sklearn.model_selection import RandomizedSearchCV
parameters = {
    'max_depth': range(2,10), 
    'max_features':stats.uniform(0.1, 1.0),
}
RS = RandomizedSearchCV(DecisionTreeClassifier(), param_distributions = parameters, cv = 10, scoring = 'accuracy', n_iter = 50)
RS.fit(x_train, y_train)

RandomizedSearchCV(cv=10, error_score=nan,
                   estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort='deprecated',
                                                    random_state=None,
         

In [27]:
RS.best_params_

{'max_depth': 4, 'max_features': 0.7086569491349372}

In [28]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(max_depth=4, max_features=0.7086)
model

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=4, max_features=0.7086, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [32]:
from sklearn.model_selection import cross_val_score
score = cross_val_score(model, x_train, y_train, scoring = 'accuracy', cv=10).mean()
score

0.8058676654182271